In [ ]:
#Librerias 
!apt-get install x11-utils > /dev/null 2>&1 
!pip install pyglet > /dev/null 2>&1 
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1

!pip install colabgymrender > /dev/null 2>&1 
#!pip install free-mujoco-py > /dev/null 2>&1 

In [ ]:
!sudo pip3 install imageio==2.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(400, 300))
display.start()

import gym
env = gym.make('MountainCar-v0')
observation = env.reset()

images = []
done = False
total_reward = 0
while not done:
  images.append(env.render(mode='rgb_array'))

  action = env.action_space.sample() # your agent here (this takes random actions)
  observation, reward, done, info = env.step(action) #transition
  total_reward += reward
env.close()

print("total reward:", total_reward)

total reward: -200.0


Creacion del video:


In [ ]:
import cv2
from moviepy.editor import *

def create_video(images, filename):
  res=(600,400) #resulotion
  out = cv2.VideoWriter(filename,cv2.VideoWriter_fourcc('M','J','P','G'), 20.0, res)
  for image in images:
      out.write(image)

  out.release()
  return VideoFileClip(filename)

clip=create_video(images, "tmp.avi")
clip.ipython_display(width=400)


Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)1744896/45929032 bytes (3.8%)3997696/45929032 bytes (8.7%)6316032/45929032 bytes (13.8%)8028160/45929032 bytes (17.5%)10608640/45929032 bytes (23.1%)13361152/45929032 bytes (29.1%)16187392/45929032 bytes (35.2%)19300352/45929032 bytes (42.0%)22315008/45929032 bytes (48.6%)25354240/45929032 bytes (55.2%)28360704/45929032 bytes (61.7%)30924800/45929032 bytes (67.3%)337

100%|█████████▉| 200/201 [00:01<00:00, 128.99it/s]


In [ ]:
# libraries
import numpy as np

state_space = 2
action_space = 3 # number of possible actions
bin_size = 30

# La tabla se inicializa con valores aleatorios entre -1 y 1
q_table = np.random.uniform(low=-1,high=1,size=([bin_size] * state_space + [action_space]))

print("Shape de tabla Q:", q_table.shape)

Shape de tabla Q: (30, 30, 3)


In [ ]:
bins = [np.linspace(-1.2,0.6,bin_size),
        np.linspace(-0.07,0.07,bin_size)]

# función para mapear el vector continuo de estados a un vector discreto
def discretize(state, bins):
    index = []
    for i in range(len(state)): index.append(np.digitize(state[i],bins[i]) - 1)
    return tuple(index)

# Ejemplo
discretize ([-1.2, 0], bins)

(0, 14)

Algoritmo Q-Learning con Epsilon = 0.001:


In [ ]:
def Q_learning(q_table, bins, episodes = 4000, gamma = 0.95, lr = 0.1, timestep = 100, epsilon = 0.01):
    #Almacenaremos las recompensas obtenidas en cada episodio
    episode_rewards = []

    visited_states = set()

    for episode in range(episodes):
        observation =  env.reset()
        episode_reward = 0
      
        done = False
        while not done:
          current_state = discretize(observation,bins)

          #1. Obtenemos acción usando epsilon-greedy policy
          if np.random.uniform(0,1) < epsilon: 
              action = env.action_space.sample()
          else:
              action = np.argmax(q_table[current_state])
          
          #2. se aplica la acción obteniendo el siguiente estado
          observation, reward, done, info = env.step(action)
          episode_reward+=reward
          next_state = discretize(observation,bins)

          if not done:
            max_future_q = np.max(q_table[next_state])
            current_q = q_table[current_state+(action,)]

            #3. Actualizamos tabla q usando Bellman
            new_q = (1-lr)*current_q + lr*(reward + gamma*max_future_q)
            q_table[current_state+(action,)] = new_q
            visited_states.add(current_state)

            current_state = next_state

        # Fin del episodio
        episode_rewards.append(episode_reward) 
        
    return episode_rewards, visited_states


In [ ]:
episode_rewards, visited_states = Q_learning(q_table, bins)

In [ ]:
print("Se accedió a "+ str(len(visited_states)) + " de los 900 estados")

In [ ]:
import matplotlib.pyplot as plt
import numpy

#graficamos promedio de 50 episodios
window_width=50
cumsum_vec = numpy.cumsum(numpy.insert(episode_rewards, 0, 0)) 
ma_vec = (cumsum_vec[window_width:] - cumsum_vec[:-window_width]) / window_width

plt.plot(ma_vec)

Creacion del video:


In [ ]:
observation =  env.reset()
episode_reward = 0

images = []
done = False
while not done:
  current_state = discretize(observation,bins)
  action = np.argmax(q_table[current_state])

  observation, reward, done, info = env.step(action)
  images.append(env.render(mode='rgb_array'))
  episode_reward+=reward
  next_state = discretize(observation,bins)

  current_state = next_state

print("reward:", episode_reward)

reward: -162.0


In [ ]:
clip=create_video(images, "tmp2.avi")
clip.ipython_display(width=400)

 99%|█████████▉| 162/163 [00:00<00:00, 238.62it/s]


In [ ]:
import tensorflow as tf
import os
import datetime
from statistics import mean
from gym import wrappers
import random
from collections import deque
from keras.models import Model, load_model
from keras.layers import Input, Dense

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
def Red(input_shape, action_space):
    X_input = Input(input_shape)

    # 'Dense' is the basic form of a neural network layer
    # Input Layer of state size(4) and Hidden Layer with 512 nodes
    X = Dense(512, input_shape=input_shape, activation="relu", kernel_initializer='he_uniform')(X_input)

    # Hidden layer with 256 nodes
    X = Dense(256, activation="relu", kernel_initializer='he_uniform')(X)
    
    # Hidden layer with 64 nodes
    X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X)

    # Output Layer with # of actions: 3 nodes 
    X = Dense(action_space, activation="softmax", kernel_initializer='he_uniform')(X)

    model = Model(inputs = X_input, outputs = X, name='MountainCar-v0')
    model.compile(loss="mse", optimizer='rmsprop', metrics=["accuracy"])

    return model

In [ ]:
class DQNAgent:
    def __init__(self):
        self.env = gym.make('MountainCar-v0')
        # by default, CartPole-v1 has max episode steps = 500
        self.state_size = self.env.observation_space.shape[0]

        # Las acciones son un vector de 3 características, 

        self.action_size = 3
        self.EPISODES = 30

        # Memoria para el experience replay
        self.memory = deque(maxlen=20000)
        
        self.gamma = 0.8   
        self.epsilon = 1.0
        self.epsilon_min = 0.0001

        # Cada iteración, el epsilon se reduce un 0.001% hasta que sea 0.001
        self.epsilon_decay = 0.999
        self.batch_size = 64

        # Comienza a entrenar en cuanto tenga 1000 experiencias
        self.train_start = 1000

        # Red principal
        self.model = Red(input_shape=(self.state_size,), action_space = self.action_size)

        # Red Objetivo
        self.target_model = Red(input_shape=(self.state_size,), action_space = self.action_size)
        self.target_model.set_weights(self.model.get_weights())

    # Guarda la experiencia y actualiza el epsilon en caso de ser necesario
    def save_experience(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        # Epsilon-greedy
        if len(self.memory) > self.train_start and self.epsilon > self.epsilon_min:
                self.epsilon *= self.epsilon_decay

    # Retorna una acción mediante epsilon-greedy
    def get_action(self, state):
        if np.random.random() <= self.epsilon:
          return random.randrange(self.action_size)
        else:
          return np.argmax(self.model.predict(state))

    def transfer_weights(self):
        self.target_model.set_weights(self.model.get_weights())

      
    # Entrenamiento mediante experience replay
    def experience_replay(self):
        if len(self.memory) < self.train_start:
            return
        # Se saca una muestra de las experiencias (subsampling)
        minibatch = random.sample(self.memory, min(len(self.memory), self.batch_size))


        state = np.zeros((self.batch_size, self.state_size))
        next_state = np.zeros((self.batch_size, self.state_size))
        action, reward, done = [], [], []


        # Se llenan las colecciones a utilizar con los datos de cada experiencia seleccionada
        for i in range(self.batch_size):
            state[i] = minibatch[i][0]
            action.append(minibatch[i][1])
            reward.append(minibatch[i][2])
            next_state[i] = minibatch[i][3]
            done.append(minibatch[i][4])

        target = self.model.predict(state)
        target_next = self.target_model.predict(next_state)

        for i in range(self.batch_size):
            # Se calculan los valores Q corregidos mediante la ecuación de Q-learning
            if done[i]:
                target[i][action[i]] = reward[i]
            else:
                target[i][action[i]] = reward[i] + self.gamma * (np.amax(target_next[i]))

        # Se entrena mediante los 
        self.model.fit(state, target, batch_size=self.batch_size, verbose=0)

    # Flujo de entrenamiento
    def run(self):
        for e in range(self.EPISODES):
            state = self.env.reset()
            state = np.reshape(state, [1, self.state_size])
            done = False
            i = 0
            cumulated_reward = 0
            while not done:
                action = self.get_action(state)
                #if para aplicar epsilon greedy  
                next_state, reward, done, info = self.env.step(action)
                #print("Recompensa: ", reward)
                next_state = np.reshape(next_state, [1, self.state_size])
                cumulated_reward = cumulated_reward + reward
                #print("Recompensa acumulada: ", cumulated_reward)
                self.save_experience(state, action, cumulated_reward, next_state, done)
                state = next_state
                i += 1
                # Transferencia de los pesos de la red principal a la objetivo
                if e % 2 == 0:
                  self.transfer_weights()
                if done:                   
                    print("episode: {}/{}, score: {}, epsilon: {:.2}, episode reward: {}".format(e, self.EPISODES, i, self.epsilon, cumulated_reward))
                self.experience_replay()

In [ ]:
agent = DQNAgent()
agent.run() 

episode: 0/30, score: 200, epsilon: 1.0, episode reward: -200.0
episode: 1/30, score: 200, epsilon: 1.0, episode reward: -200.0
episode: 2/30, score: 200, epsilon: 1.0, episode reward: -200.0
episode: 3/30, score: 200, epsilon: 1.0, episode reward: -200.0
episode: 4/30, score: 200, epsilon: 1.0, episode reward: -200.0
episode: 5/30, score: 200, epsilon: 0.82, episode reward: -200.0
episode: 6/30, score: 200, epsilon: 0.67, episode reward: -200.0
episode: 7/30, score: 200, epsilon: 0.55, episode reward: -200.0
episode: 8/30, score: 200, epsilon: 0.45, episode reward: -200.0
episode: 9/30, score: 200, epsilon: 0.37, episode reward: -200.0
episode: 10/30, score: 200, epsilon: 0.3, episode reward: -200.0
episode: 11/30, score: 200, epsilon: 0.25, episode reward: -200.0
episode: 12/30, score: 200, epsilon: 0.2, episode reward: -200.0
episode: 13/30, score: 200, epsilon: 0.17, episode reward: -200.0
episode: 14/30, score: 200, epsilon: 0.14, episode reward: -200.0
episode: 15/30, score: 200,

In [ ]:
import gym
from colabgymrender.recorder import Recorder

env = gym.make('MountainCar-v0')
directory = './video'
env = Recorder(env, directory)

# Para que siempre obtenga la máxima recompensa en el testing
agent.epsilon = -1
observation = env.reset()
terminal = False
while not terminal:
  state = np.reshape(observation, [1, agent.state_size])
  action = agent.get_action(state)
  observation, reward, terminal, info = env.step(action)

print("Reward: ", reward)
env.play()

Reward:  -1.0
